In [1]:
from dataloader import *
from  config import *
from data_processor import preprocess_data

Using TensorFlow backend.


In [2]:
data = DataLoader(config_mixed_flickr8k_flickr8k_n)

Loading dataset
Loading train dataset
Loading images splits
Images splits loaded
Number of train images:  6000
Number of test images:  1000
Loading all captions
All captions loaded
Nuber of all captions:  8092
Loading captions splits
Captions splits loaded
Number of train captions:  6000
Number of test test:  1000
Loading test dataset
Loading images splits
Images splits loaded
Number of train images:  6000
Number of test images:  1000
Loading all captions
All captions loaded
Nuber of all captions:  8092
Loading captions splits
Captions splits loaded
Number of train captions:  6000
Number of test test:  1000


In [3]:
data=preprocess_data(data)

Number of train images:  6000
Number of test images:  1000
Number of train captions:  6000
Number of test captions:  1000
Descriptions cleaned.
[['a', 'group', 'of', 'people', 'are', 'partying', 'at', 'a', 'masquerade', 'party'], ['a', 'lady', 'in', 'red', 'and', 'black', 'grins', 'at', 'the', 'camera', 'at', 'a', 'costume', 'party'], ['a', 'woman', 'in', 'a', 'domino', 'mask', 'and', 'severe', 'hair', 'is', 'at', 'a', 'party'], ['a', 'woman', 'in', 'a', 'red', 'dress', 'and', 'black', 'mask', 'is', 'on', 'a', 'crowded', 'dance', 'floor'], ['a', 'woman', 'wears', 'a', 'red', 'dress', 'and', 'a', 'black', 'mask', 'while', 'people', 'dance', 'behind', 'her']]
Descriptions wraped into start and stop words.
['START a child in a pink dress is climbing up a set of stairs in an entry way STOP', 'START a girl going into a wooden building STOP', 'START a little girl climbing into a wooden playhouse STOP', 'START a little girl climbing the stairs to her playhouse STOP', 'START a little girl in a

2022-09-09 20:14:55.571085: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-09 20:14:55.571661: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 16. Tune using inter_op_parallelism_threads for best performance.


Processed:
100
Processed:
200
Processed:
300
Processed:
400
Processed:
500
Processed:
600
Processed:
700
Processed:
800
Processed:
900
Processed:
1000
Test images encoded
Time taken in seconds = 271.31609773635864
Encoded test images saved under 
./mixed_flickr8k_8k_n/Pickle/encoded_test_images.pkl
Processed:
100
Processed:
200
Processed:
300
Processed:
400
Processed:
500
Processed:
600
Processed:
700
Processed:
800
Processed:
900
Processed:
1000
Processed:
1100
Processed:
1200
Processed:
1300
Processed:
1400
Processed:
1500
Processed:
1600
Processed:
1700
Processed:
1800
Processed:
1900
Processed:
2000
Processed:
2100
Processed:
2200
Processed:
2300
Processed:
2400
Processed:
2500
Processed:
2600
Processed:
2700
Processed:
2800
Processed:
2900
Processed:
3000
Processed:
3100
Processed:
3200
Processed:
3300
Processed:
3400
Processed:
3500
Processed:
3600
Processed:
3700
Processed:
3800
Processed:
3900
Processed:
4000
Processed:
4100
Processed:
4200
Processed:
4300
Processed:
4400
Proce

In [4]:
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector, \
    Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers.merge import add
from keras.models import Model
from keras import Input, layers
from keras import callbacks
from eval_utils import calculate_results, prepare_for_evaluation
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from numpy import array
import pickle
from eval_utils import generate_report

In [5]:
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch, vocab_size):
    """
    Data generator, that serves the data to the model during training


    Parameters
    ----------
    descriptions: str
        Dictionary where key is image id and value is a list of preprocessed descriptions
    photos
        Dictionary with encoded images(vector of image features extracted by specified image feature extractor
         fe. Inception), identified by image id
    wordtoix
        Dictionary with keys-words , values -id of word
    max_length
        Max number of words in caption on dataset
    num_photos_per_batch: int
    vocab_size: int
    Returns
    -------
    """
    X1, X2, y = list(), list(), list()
    n = 0
    # loop for ever over images
    while 1:
        for key, desc_list in descriptions.items():
            n += 1
            # retrieve the photo feature from the dictionary
            photo = photos[key]
            for desc in desc_list:
                # encode the sentence by translating it to the number representation,
                # with the dictionary of words created in in the previous stage
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # split one sequence into multiple X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # store
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # yield the batch data
            if n == num_photos_per_batch:
                yield ([array(X1), array(X2)], array(y))
                X1, X2, y = list(), list(), list()
                n = 0

class ModelImpl:
    def __init__(self, data):
        self.data=data
        inputs1 = Input(shape=(2048,))
        fe1 = Dropout(0.5)(inputs1)
        fe2 = Dense(256, activation='relu')(fe1)
        inputs2 = Input(shape=(self.data.max_length,))
        se1 = Embedding(self.data.vocab_size, general[self.data.language]["embedings_dim"], mask_zero=True)(inputs2)
        se2 = Dropout(0.5)(se1)
        se3 = LSTM(256)(se2)
        decoder1 = add([fe2, se3])
        decoder2 = Dense(256, activation='relu')(decoder1)
        outputs = Dense(self.data.vocab_size, activation='softmax')(decoder2)
        self.model = Model(inputs=[inputs1, inputs2], outputs=outputs)
        self.model.summary()
        self.model.layers[2]

        self.model.layers[2].set_weights([self.data.embedding_matrix])
        self.model.layers[2].trainable = False

        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer())
        self.setup()

    def optimizer(self):
        return Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    def setup(self):
        # model.optimizer.lr = 0.001
        self.epochs = 2
        self.number_pics_per_bath = 100
        self.steps = len(self.data.train_captions_wrapped) // self.number_pics_per_bath

    def train(self):
        model_weights_path="./" + self.data.configuration["data_name"] + self.data.configuration["model_save_dir"]
        if self.data.configuration["train_model"]:
            callback = callbacks.EarlyStopping(monitor='loss', min_delta=0.001, patience=3)
            generator = data_generator(self.data.train_captions_wrapped,
                                       self.data.encoded_images_train,
                                       self.data.wordtoix,
                                       self.data.max_length,
                                       self.number_pics_per_bath,
                                       self.data.vocab_size)
            self.model.fit(generator, epochs=self.epochs,
                           steps_per_epoch=self.steps,
                           callbacks=[callback],
                           verbose=1)
            if self.data.configuration["save_model"]:
                writepath = model_weights_path+ "/"+'model' + '.h5'
                self.model.save(writepath)
                self.model.save_weights(model_weights_path
                                        + self.data.configuration["model_save_path"])
        else:
            self.model.load_weights(model_weights_path
                                        +self.data.configuration["model_save_path"])

    def evaluate(self):
        expected, results = prepare_for_evaluation(self.data.encoded_images_test, self.data, self.model)
        out = calculate_results(expected, results, self.data.configuration)
        print(out)

model=ModelImpl(data)


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 37)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2048)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 37, 199)      329345      input_3[0][0]                    
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 2048)         0           input_2[0][0]                    
____________________________________________________________________________________________

In [6]:
model.train()

Epoch 1/2
60/60 [==============================] - 497s 8s/step - loss: 5.0682
Epoch 2/2
60/60 [==============================] - 482s 8s/step - loss: 4.0723


/Users/mateuszb/opt/anaconda3/envs/framework/lib/python3.7/site-packages/keras/engine/saving.py:165: UserWarning: TensorFlow optimizers do not make it possible to access optimizer attributes or optimizer state after instantiation. As a result, we cannot save the optimizer as part of the model save file.You will have to compile your model again after loading it. Prefer using a Keras optimizer instead (see keras.io/optimizers).
  'TensorFlow optimizers do not '


In [7]:
model.evaluate()

Preparing for evaluation


  0%|          | 0/1000 [00:00<?, ?it/s]

Processed:
0
Processed:
100
Processed:
200
Processed:
300
Processed:
400
Processed:
500
Processed:
600
Processed:
700
Processed:
800
Processed:
900
tokenization...


PTBTokenizer tokenized 64178 tokens at 332638.84 tokens per second.
PTBTokenizer tokenized 19766 tokens at 188432.80 tokens per second.


setting up scorers...
computing Bleu score...
{'testlen': 18767, 'reflen': 12348, 'guess': [18767, 17767, 16767, 15767], 'correct': [5485, 1789, 395, 103]}
ratio: 1.5198412698411468
Bleu_1: 0.292
Bleu_2: 0.172
Bleu_3: 0.089
Bleu_4: 0.046
computing METEOR score...
METEOR: 0.127
computing Rouge score...
ROUGE_L: 0.328
computing CIDEr score...
CIDEr: 0.142
computing SPICE score...


Parsing reference captions
Initiating Stanford parsing pipeline
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.TokenizerAnnotator - TokenizerAnnotator: No tokenizer type provided. Defaulting to PTBTokenizer.
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ssplit
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator parse
[main] INFO edu.stanford.nlp.parser.common.ParserGrammar - Loading parser from serialized file edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz ... 
done [0.5 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
Loading classifier from edu/stanford/nlp/models/ner/english.all.3class.distsim.crf.ser.gz ... done [0.9 sec].
Loading classifier from edu/stanford/nlp/models/ner/english.muc.7class.distsim.crf.ser.gz ... done [0.5 sec].
Loading classif

SPICE evaluation took: 1.513 min
SPICE: 0.076
computing WMD score...
WMD: 0.383


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'Bleu_1': 0.2922683433686635, 'Bleu_2': 0.17154932282882232, 'Bleu_3': 0.08850610918173352, 'Bleu_4': 0.046131967365310785, 'METEOR': 0.12673418954682494, 'ROUGE_L': 0.32848613308537294, 'CIDEr': 0.14162510438388648, 'SPICE': 0.07603207168682966, 'WMD': 0.38274339838459465}


In [8]:
generate_report(general["results_directory"])



 Final results saved to final_results.csv
